# DEMO de Elaboración y Ejecución de Benchmark

In [ ]:
# Poned vuestra API KEY de groq aquí:
groq_api_key = "[removed for privacy reasons]" #add yours

###   Clase Benchmark + Groq

In [6]:
%pip install groq tabulate
%pip install requests
import json, re, requests
from groq import Groq
from tabulate import tabulate

class LLMBenchmark:
    def __init__(self, groq_api_key, models = ["llama-3.2-90b-vision-preview", "llama-3.3-70b-versatile", "mixtral-8x7b-32768", "gemma2-9b-it", "llama-3.2-3b-preview"]):
        self.groq_api_key = groq_api_key
        self.groq_client = Groq(api_key=groq_api_key)
        self.questions = []
        self.models = models

    def add_question(self, question, options, correct_answer):
        # Formateamos las opciones con letras
        formatted_options = {chr(65 + i): option for i, option in enumerate(options)}
        formatted_options_inverse = {option : chr(65 + i) for i, option in enumerate(options)}
        self.questions.append({
            "question": question,
            "options": formatted_options,
            "correct_answer": formatted_options_inverse[correct_answer] if correct_answer in formatted_options_inverse else correct_answer
        })

    def print_groq_models(self):
        url = "https://api.groq.com/openai/v1/models"
        headers = {
            "Authorization": f"Bearer {self.groq_api_key}",
            "Content-Type": "application/json"
        }
        response = requests.get(url, headers=headers)
        print(response.json())

    def add_model(self, *model_names):
        self.models.extend(model_names)

    def print_debug(self, message, debug):
        if debug:
            print(message)

    def run_benchmark(self, debug=False):
        results = {}
        for model_name in self.models:
            correct_predictions = 0
            for question_data in self.questions:
                question = question_data['question']
                options = question_data['options']
                correct_answer = question_data['correct_answer']

                # Construimos el prompt con las opciones
                options_text = "\n".join([f"{label}: {option}" for label, option in options.items()])
                prompt = f"{question}\nOpciones:\n{options_text}\n\nPor favor, elige la letra de la opción correcta y responde solo con esa letra y nada más."

                try:
                    chat_completion = self.groq_client.chat.completions.create(
                        messages=[{"role": "user", "content": prompt}],
                        model=model_name,
                        max_tokens=50,
                        temperature=0,
                    )
                    model_answer = chat_completion.choices[0].message.content.strip().upper()

                    # Validar la respuesta con regex
                    if correct_answer.upper() in model_answer:
                        correct_predictions += 1
                        self.print_debug(f"✅ Model: {model_name} | Question: {question} | Answer: {model_answer} | Correct: {correct_answer}", debug)
                    else:
                        self.print_debug(f"❌ Model: {model_name} | Question: {question} | Answer: {model_answer} | Correct: {correct_answer}", debug)
                except Exception as e:
                    print(f"Error querying {model_name}: {e}")

            # Calcula la tasa de aciertos
            total_questions = len(self.questions)
            accuracy = correct_predictions / total_questions if total_questions else 0
            results[model_name] = (correct_predictions, total_questions, accuracy)

        # Imprimir resultados en una tabla
        headers = ["Modelo", "Respuestas correctas", "Exactitud"]
        table = [[model, f"{correct}/{total}", f"{accuracy:.2%}"] for model, (correct, total, accuracy) in results.items()]
        print(tabulate(table, headers=headers, tablefmt='grid'))
        #return results

--- Logging error ---
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/Library/

###   Ejemplos

In [7]:
llm_benchmark_test = LLMBenchmark(groq_api_key)
llm_benchmark_test.add_question("¿Cuál es la capital de Francia?", ["París", "Roma", "Berlín"], "A")
llm_benchmark_test.add_question("¿Cuál es el río más largo del mundo?", ["Nilo", "Amazonas", "Danubio"], "Nilo")
llm_benchmark_test.add_question("¿En qué año se descubrió América?", ["1492", "1776", "1642"], "1492")
llm_benchmark_test.add_question("¿Cuál es la fórmula química del agua?", ["H2O", "CO2", "NaCl"], "A")
llm_benchmark_test.run_benchmark(True)

✅ Model: llama-3.2-90b-vision-preview | Question: ¿Cuál es la capital de Francia? | Answer: A | Correct: A
✅ Model: llama-3.2-90b-vision-preview | Question: ¿Cuál es el río más largo del mundo? | Answer: A | Correct: A
✅ Model: llama-3.2-90b-vision-preview | Question: ¿En qué año se descubrió América? | Answer: A | Correct: A
✅ Model: llama-3.2-90b-vision-preview | Question: ¿Cuál es la fórmula química del agua? | Answer: A | Correct: A
✅ Model: llama-3.3-70b-versatile | Question: ¿Cuál es la capital de Francia? | Answer: A | Correct: A
✅ Model: llama-3.3-70b-versatile | Question: ¿Cuál es el río más largo del mundo? | Answer: A | Correct: A
✅ Model: llama-3.3-70b-versatile | Question: ¿En qué año se descubrió América? | Answer: A | Correct: A
✅ Model: llama-3.3-70b-versatile | Question: ¿Cuál es la fórmula química del agua? | Answer: A | Correct: A
Error querying mixtral-8x7b-32768: Error code: 400 - {'error': {'message': 'The model `mixtral-8x7b-32768` has been decommissioned and is 

In [ ]:
llm_benchmark_test2 = LLMBenchmark(groq_api_key)
llm_benchmark_test2.add_question("¿Cuál es el elemento químico más abundante en la corteza terrestre?", ["Oxígeno", "Silicio", "Aluminio"], "Oxígeno")
llm_benchmark_test2.run_benchmark(True)

✅ Model: llama-3.2-90b-vision-preview | Question: ¿Cuál es el elemento químico más abundante en la corteza terrestre? | Answer: A | Correct: A
❌ Model: llama-3.2-90b-vision-preview | Question: ¿Cuál es la derivada de x^2? | Answer: A | Correct: 5x
❌ Model: llama-3.2-90b-vision-preview | Question: Explica el concepto de inflación. | Answer: A | Correct: Variaciones en el tipo de cambio.
❌ Model: llama-3.2-90b-vision-preview | Question: Explica el concepto de inflación. | Answer: A | Correct: Variaciones en el tipo de cambio.
❌ Model: llama-3.2-90b-vision-preview | Question: Explica el concepto de inflación. | Answer: A | Correct: Variaciones en el tipo de cambio.
✅ Model: llama-3.3-70b-versatile | Question: ¿Cuál es el elemento químico más abundante en la corteza terrestre? | Answer: A | Correct: A
❌ Model: llama-3.3-70b-versatile | Question: ¿Cuál es la derivada de x^2? | Answer: A | Correct: 5x
❌ Model: llama-3.3-70b-versatile | Question: Explica el concepto de inflación. | Answer: A 

#   CÓDIGO PRÁCTICA

In [23]:
llm_benchmark_test2 = LLMBenchmark(groq_api_key)
llm_benchmark_test2.add_question("Analiza la complejidad computacional del algoritmo de Dijkstra para encontrar el camino más corto en un grafo ponderado, considerando diferentes estructuras de datos para representar el grafo (listas de adyacencia, matrices de adyacencia) y la implementación de la cola de prioridad.", ["Complejidad de O(E log V), siendo E el número de aristas y V el número de vértices del grafo. Dependiendo de la implementación de la cola de prioridad y la representación del grafo.", "Complejidad de O(V^2) con matriz de adyacencia, O(E log V) con lista de adyacencia y cola de prioridad.", "Algoritmo con complejidad polinomial, pero su eficiencia depende de la elección de la cola de prioridad y del tamaño del grafo.","La complejidad depende de la distribución de los pesos y la cantidad de nodos del grafo."], "A")
llm_benchmark_test2.run_benchmark(True)

❌ Model: llama-3.2-90b-vision-preview | Question: Analiza la complejidad computacional del algoritmo de Dijkstra para encontrar el camino más corto en un grafo ponderado, considerando diferentes estructuras de datos para representar el grafo (listas de adyacencia, matrices de adyacencia) y la implementación de la cola de prioridad. | Answer: B | Correct: A
❌ Model: llama-3.3-70b-versatile | Question: Analiza la complejidad computacional del algoritmo de Dijkstra para encontrar el camino más corto en un grafo ponderado, considerando diferentes estructuras de datos para representar el grafo (listas de adyacencia, matrices de adyacencia) y la implementación de la cola de prioridad. | Answer: B | Correct: A
Error querying mixtral-8x7b-32768: Error code: 400 - {'error': {'message': 'The model `mixtral-8x7b-32768` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invali